<a href="https://colab.research.google.com/github/quangminh141005/Learn_PyTorch/blob/main/60_minutes_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
x_data, x_data.dtype

(tensor([[1, 2],
         [3, 4]]),
 torch.int64)

In [ ]:
new_data = [[1., 2.], [3., 4.]]
np_array = np.array(new_data)
tensor_np = torch.from_numpy(np_array)
tensor_np, tensor_np.dtype
# np_array, np_array.dtype

(tensor([[1., 2.],
         [3., 4.]], dtype=torch.float64),
 torch.float64)

###**From another tensor:**
* The new tensor reatins the properties (shpae, datatype) of the argument tensor, unless explicitly overidden

In [ ]:
x_ones = torch.ones_like(x_data)
x_ones

tensor([[1, 1],
        [1, 1]])

In [ ]:
tensor_rand = torch.rand_like(x_data,dtype=torch.float)
tensor_rand

tensor([[0.8348, 0.7478],
        [0.9953, 0.9918]])

In [ ]:
shape = (2, 3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Randow tensor in {shape} shape: {rand_tensor}")
print(f"Ones tensor in {shape} shape: {ones_tensor}")
print(f"Zeros tensor in {shape} shape: {zeros_tensor}")

Randow tensor in (2, 3) shape: tensor([[0.0400, 0.5581, 0.8330],
        [0.4348, 0.1213, 0.7872]])
Ones tensor in (2, 3) shape: tensor([[1., 1., 1.],
        [1., 1., 1.]])
Zeros tensor in (2, 3) shape: tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
print(f"Device tensor is stored: {rand_tensor.device}")

Device tensor is stored: cpu


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

## PyTorch autograd

In [ ]:
x = torch.tensor([3.0], requires_grad=True)

y = x**2

y.backward()

print(x.grad)

tensor([6.])


In [ ]:
# Example of a pretrained model
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model class
class SimpleNet(nn.Module):
  def __init__(self):
    super(SimpleNet, self).__init__()
    # Define layers
    self.fc1 = nn.Linear(784, 128) # input layer (28x28 pixels flatten to 784)
    self.fc2 = nn.Linear(128, 64) # hidden layer
    self.fc3 = nn.Linear(64, 10) # Output layer (10 classes)

  def forward(self, x):
    # Forward pass logic
    x = F.relu(self.fc1(x)) # pass through the first layer + ReLU
    x = F.relu(self.fc2(x)) # pass through second layer + ReLU
    x = self.fc3(x)         # final layer, raw scores (logits)
    return x

In [ ]:
model_simple = SimpleNet()
model_simple

SimpleNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
input_data = torch.rand(1, 784)

In [ ]:
output = model_simple(input_data)
output.shape

torch.Size([1, 10])

In [ ]:
output

tensor([[ 0.1267,  0.0366, -0.0011, -0.1782, -0.0930, -0.0457, -0.0986, -0.1442,
         -0.0266, -0.1835]], grad_fn=<AddmmBackward0>)

In [ ]:
prediction = model(data)

In [ ]:
loss = (prediction - labels).sum()
loss.backward()

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
optim.step()

In [ ]:
# Another example of .backward()
import torch
import torch.nn as nn
import torch.optim as optim

# Create some dummy imput and target
x = torch.tensor([[1.0], [2.0], [3.0]], requires_grad=False)
y = torch.tensor([[2.0], [4.0], [6.0]], requires_grad=False)

# Define a simple linear model: y = ax+ b
model = nn.Linear(1, 1) # One input, one putput


# Define loss function: Mean Squarred Error
criterion = nn.MSELoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Forward pass
y_pred = model(x)
loss = criterion(y_pred, y)
print(f"Loss before backward: {loss.item()}")

# Backward pass
loss.backward()

for name, param in model.named_parameters():
  if param.requires_grad:
    print(f"Gradient for {name}: {param.grad}")

optimizer.step()
optimizer.zero_grad()


Loss before backward: 5.772523403167725
Gradient for weight: tensor([[-10.3005]])
Gradient for bias: tensor([-4.1896])


## Differentiation in Autograd

In [ ]:
import torch
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

In [ ]:
Q = 3*a**3 - b**2

In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [ ]:
# A simple use of backward
import torch

x = torch.tensor(2.0, requires_grad=True)

y = x**2

y.backward()

print(x.grad)

tensor(4.)


In [ ]:
# Example with enternal grad (when the output is not a scalar)

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

y = x*2

external_grad = torch.tensor([0.1, 1.0, 0.0001])

y.backward(gradient=external_grad)

print(x.grad)

tensor([2.0000e-01, 2.0000e+00, 2.0000e-04])


NameError: name 'loss' is not defined

## Neural Networks

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    # 1 input image, 6 output channels, 5x5 square convolution
    # Kernel
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    # y = Wx + b
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, input):
    # conv1: 1 input channel, 6 output channels
    # 5x5 convolution, it uses RELU activation, and outputs
    # a Tensor with size (N, 8, 28, 28), where N is the size
    # of the batch
    c1 = F.relu(self.conv1(input))

    # Subsampling layer S2: 2x2 grid, purely functional,
    # this layer doesn't have any parameters (because it
    # picking the max value from max_pool2d)
    s2 = F.max_pool2d(c1, (2, 2))

    # Convolotion layer C3: 6 input channels, 16 output channels
    # 5x5 convolution, use ReLU activation
    c3 = F.relu(self.conv2(s2))

    # Sampling layer: 2x2 grid
    s4 = F.max_pool2d(c3, 2)

    # Flatten the tensor, ouput a (N, 400) Tensor
    s4 = torch.flatten(s4, 1)

    # input: (N, 400)
    # Output: (N, 120)
    f5 = F.relu(self.fc1(s4))

    f6 = F.relu(self.fc2(f5))

    output = self.fc3(f6)
    return output

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [13]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0383, -0.0873, -0.0243, -0.0684,  0.0417,  0.0082,  0.0045, -0.0624,
         -0.0023, -0.0460]], grad_fn=<AddmmBackward0>)


In [14]:
net.zero_grad() # Clear the previous gradient for the next gradient descent
out.backward(torch.randn(1, 10))

In [15]:
# Compute the loss function
output = net(input)
target = torch.randn(10) # a dummy target
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6942, grad_fn=<MseLossBackward0>)


In [16]:
# Illustration of loss.backward()
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [17]:
# Backprop
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0014, -0.0027,  0.0104,  0.0065, -0.0087, -0.0089])


In [20]:
# Update the weight using gradient descent (SGD)

# learning_rate = 0.01
# for f in net.parameters():
  # f.data.sub_(f.grad.data * learning_rate)

# Create optimizer
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)

# In training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()